In [2]:
import sys
import os
import glob
sys.path.append('/Volumes/T7/Software')
from cardpy.Sample_Data.Data_Paths import *
from cardpy.Data_Import            import *
from cardpy.Data_Sorting           import *
from cardpy.Data_Processing.DTI    import *
from cardpy.GUI_Tools              import IntERCOMS
from cardpy.Colormaps              import *
from cardpy.Data_Saving            import *
from cardpy.Data_Processing.cDTI   import *
import matplotlib.pyplot as plt
import numpy as np
import scipy.io

cDTI_cmaps     = cDTI_Colormaps_Generator()

In [3]:
main_path                = sample_Output_Folder()
NifTi_path               = glob.glob(os.path.join(main_path, '*Interpolated', '*.nii'))[0]
header_path              = glob.glob(os.path.join(main_path, '*Interpolated', '*.header'))[0]
b_values_path            = glob.glob(os.path.join(main_path, '*Interpolated', '*.bvals'))[0]
b_vectors_path           = glob.glob(os.path.join(main_path, '*Interpolated', '*.bvecs'))[0]

Segmentaiton_Path        = os.path.join(main_path, '11_Segmentation')
if os.path.isdir(Segmentaiton_Path) == False:
    os.makedirs(Segmentaiton_Path)
Quantiative_Results_Path = os.path.join(main_path, '12_Quantitative_Results')
if os.path.isdir(Quantiative_Results_Path) == False:
    os.makedirs(Quantiative_Results_Path)

[matrix_stacked, b_vals_stacked, b_vecs_stacked, Header, _, _] = NifTi_Reader(NifTi_path, b_values_path, b_vectors_path, header_path)
[matrix_sorted,  b_vals_sorted,  b_vecs_sorted]                = stacked2sorted(matrix_stacked, b_vals_stacked, b_vecs_stacked)
[_, _, Eigenvectors, Standard_DTI_Metrics]                     = DTI_recon(matrix_sorted, b_vals_sorted, b_vecs_sorted, tensor_fit = 'NLLS')
#[Endo_Centers, Endo_Axes, Epi_Centers, Epi_Axes]               = IntERCOMS.IntERCOMS_GUI(matrix_sorted[:, :, :, 0], Standard_DTI_Metrics['MD'], Eigenvectors['E1'], Line_Width = 1)
#[myocardium_mask, NRRD2, NRRD1]                                = IntERCOMS.IntERCOMS_Mask_Making(matrix_sorted, Endo_Centers, Endo_Axes, Epi_Centers, Epi_Axes)
#Save_NRRD_Segmentation(NRRD1, Header, Segmentaiton_Path, 'LV_Blood_Pool')
#Save_NRRD_Segmentation(NRRD2, Header, Segmentaiton_Path, 'LV_Myocardium+Blood_Pool')

myocardial_mask              = myocardium_mask
# image = myocardial_mask
num_interp_points            = 200
#Tyler -> changed from medium to native, so it doesn't interoplate the mask, and doesn't smooth it out, double check:... tbd
#Should get the raw masks, because trainnig is done on smoothed images so we compare the two outputs on the raw afterwards!
smoothness_level             = 'Native'
Helix_Angle_Filter_Settings  = dict()
Helix_Angle_Filter_Settings['Linear Filter: Outlier StDev']      = 1
Helix_Angle_Filter_Settings['Spatial Filter: Wall Depth Factor'] = 0.25
Helix_Angle_Filter_Settings['Spatial Filter: Kernel Size']       = 5


#mask is either GT or predicted!!, so need to run one for GT and one for pred!
#split here... eg. make function out of above here, and then do the same for GT and prediciton.
#....


[Cardiac_DTI_Metrics, Epi, Endo, Mask] = cDTI_recon(myocardial_mask, Eigenvectors, num_interp_points, smoothness_level, Helix_Angle_Filter_Settings)

myocardial_mask_smoothed = np.copy(Mask)
#if native, smoothness and regular one should be equal
if (myocardial_mask == myocardial_mask_smoothed):
    print('smoothness level is set to native, so check should pass!')

else:
    print('some smoothenss is happneing on the masks.... double check')
    #maybe set to low/ medium,  otherwise save as a new smoothed or something.. as an 07 folder in the orignal folder strucutre and then do the HAP assesmetnnow    
    #if this trigers we have ot use the smoothed one for MD FA etc. too!
    #could also save this as excel.. first check if trigger even happens.


myocardial_mask_smoothed_nan = np.copy(myocardial_mask_smoothed)
myocardial_mask_smoothed_nan = myocardial_mask_smoothed_nan.astype('float')

myocardial_mask_smoothed_nan [myocardial_mask_smoothed_nan == 0] = np.nan

grid     = Endo2Epi_Grid(myocardial_mask_smoothed)
grid_nan = np.copy(grid)
grid_nan = grid_nan * myocardial_mask_smoothed_nan
grid_nan = np.clip(grid_nan, 0.0, 1)
matrix   = matrix_sorted

#what we want to save:
#Standard_DTI_Metrics -> mat file, stored as a dictionary atm.
#Cardiac_DTI_Metrics -> ^
#for both GT and predicted!!! so need two!

IndexError: list index out of range

In [5]:
main_path                = sample_Output_Folder()

main_path = '/Users/saschastocker/Documents/Stanford/work2024/Paper2025Automatic/HAanalysis'

for volunteer in os.listdir(main_path):
    for dataset in os.listdir(os.path.join(main_path,volunteer,'Distortion_Corrected')):
        dataset

    
NifTi_path               = glob.glob(os.path.join(main_path, '*Interpolated', '*.nii'))[0]
header_path              = glob.glob(os.path.join(main_path, '*Interpolated', '*.header'))[0]
b_values_path            = glob.glob(os.path.join(main_path, '*Interpolated', '*.bvals'))[0]
b_vectors_path           = glob.glob(os.path.join(main_path, '*Interpolated', '*.bvecs'))[0]
#map to the data we pull in from the orignal images from folders
input_path_data = ""

#use for later integration in the segmentaiotn analysis
input_path_segmentation = ""


FileNotFoundError: [Errno 2] No such file or directory: '/Users/saschastocker/Documents/Stanford/work2024/Paper2025Automatic/HAanalysis'

In [7]:
from sklearn.linear_model import LinearRegression


HAP_gt = []
HAR_gt = []

for slc in range(myocardial_mask_smoothed_nan.shape[2]):
    E2E_data = grid_nan[:, :, slc].flatten()
    E2E_data = E2E_data[~(np.isnan(grid_nan[:, :, slc].flatten()))]
    x = E2E_data
    xi = np.copy(x)



    HA_data  = Cardiac_DTI_Metrics['HA'][:, :, slc].flatten()
    HA_data  = HA_data[~(np.isnan(grid_nan[:, :, slc].flatten()))]
    y = HA_data
    yi = np.copy(y)

    # Calculate the point density
    xy = np.vstack([x, y])
    z = gaussian_kde(xy)(xy)

    # Sort the points by density, so that the densest points are plotted last
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]
    density = plt.scatter(x, y, c=z, s=50, cmap = 'hot')


    model                 = LinearRegression().fit(x[:, np.newaxis], y[:, np.newaxis])
    #enforcing true 0 and true 1, true epi and endo endpoint/limit
    x_predicted = np.linspace(0,1, 101)
    y_predicted = model.intercept_ + model.coef_ * x_predicted[:, np.newaxis]

    #adds the helix angle pitches from each slice
    HAP_gt.append(model.coef_)

    #get the ranges, positive - (negative) -> range of the angles.
    HAR_gt.append(y_predicted[0]-y_predicted[-1])


# #repeat for predicted mask TODO!
# HAP_pred..
# HAR_pred....
#compare -> similar to MD and FA maps comparison... should be super close otherwise the segmentation is off
#can compare the pitches or the ranges, depending on what's more tellling/ better to look at.

NameError: name 'myocardial_mask_smoothed_nan' is not defined

In [10]:
x_predicted = np.linspace(0,1, 101)
x_predicted

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])